In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import dayofweek

In [0]:
 # Azure storage access info
blob_account_name = "bgupb202402juanbarriento"
blob_container_name = "marketplace"
blob_sas_token = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-12-01T01:56:16Z&st=2024-10-16T17:56:16Z&spr=https&sig=cM7vK4vvrL9YH31ZYI%2BX%2BmssMUTdRieP%2BMGcVNPoagA%3D'

# Allow SPARK to read from Blob remotely
wasbs_path = f'wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/'
spark.conf.set( f'fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net' , blob_sas_token)
print('Remote blob path: ' + wasbs_path)

Remote blob path: wasbs://marketplace@bgupb202402juanbarriento.blob.core.windows.net/


In [0]:
bronze_df = spark.sql("SELECT * FROM jdbr_mkp.mkp_bronze").cache()

In [0]:
bronze_df = bronze_df.drop("event_time")

In [0]:
silver_df = bronze_df.withColumn('day_of_week', dayofweek('event_time_timestamp'))

In [0]:
display(silver_df.head(10))

event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category,subcategory,month,year,event_time_timestamp,day_of_week
view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,appliances,environment,10,2019,2019-10-01T00:00:00Z,3
view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook,10,2019,2019-10-01T00:00:01Z,3
view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone,10,2019,2019-10-01T00:00:04Z,3
view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,computers,desktop,10,2019,2019-10-01T00:00:05Z,3
view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,apparel,shoes,10,2019,2019-10-01T00:00:10Z,3
view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,electronics,smartphone,10,2019,2019-10-01T00:00:11Z,3
view,2900536,2053013554776244595,appliances.kitchen.microwave,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd,appliances,kitchen,10,2019,2019-10-01T00:00:11Z,3
view,1005011,2053013555631882655,electronics.smartphone,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101,electronics,smartphone,10,2019,2019-10-01T00:00:11Z,3
view,3900746,2053013552326770905,appliances.environment.water_heater,haier,102.38,555444559,98b88fa0-d8fa-4b9d-8a71-3dd403afab85,appliances,environment,10,2019,2019-10-01T00:00:13Z,3
view,13500240,2053013557099889147,furniture.bedroom.bed,brw,93.18,555446365,7f0062d8-ead0-4e0a-96f6-43a0b79a2fc4,furniture,bedroom,10,2019,2019-10-01T00:00:16Z,3


In [0]:
silver_df.write.format("delta").mode("overwrite").partitionBy("year", "month", "category", "subcategory").save(f"{wasbs_path}silver")

In [0]:
%sql
USE jdbr_mkp;
DROP TABLE IF EXISTS mkp_silver;
CREATE TABLE mkp_silver
USING DELTA
LOCATION 'wasbs://marketplace@bgupb202402juanbarriento.blob.core.windows.net/silver/'

In [0]:
%sql
SELECT * FROM jdbr_mkp.mkp_silver
LIMIT 10;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: Unable to access container marketplace in account bgupb202402camilosoto.blob.core.windows.net using anonymous credentials, and no credentials found for them  in the configuration.
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1167)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:551)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1392)
	at com.databricks.common.filesystem.LokiFileSystem$.getLokiFS(LokiFileSystem.scala:186)
	at com.databricks.common.filesystem.LokiFileSystem.initialize(LokiFileSystem.scala:257)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSyste